<a href="https://colab.research.google.com/github/Giovanna-de-Oliveira/LH_CD_GIOVANNADASILVADEOLIVEIRA/blob/main/modeling_and_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão da Nota do IMDB

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle


A escolha de um modelo de regressão linear para a predição de notas IMDB se deve pelas seguntes razões:

1. **Interpretabilidade** -> A regressão linear fornece coeficientes que podem ser interpretados diretamente. Isso significa que é possível entender como cada variável de entrada contribui para a previsão da nota do IMDB.

2.  **Simplicidade** -> É um modelo relativamente simples e fácil de entender, o que pode ser vantajoso para explicar os resultados a stakeholders não técnicos.

3. **Eficiência computacional** -> Treinar e usar um modelo de regressão linear é computacionalmente mais eficiente em comparação com modelos mais complexos, o que pode ser importante ao lidar com grandes conjuntos de dados e treinamentos em ambientes mais computacionalmente limitados.

4. **Boa performance inicial** -> Em muitos casos, um modelo de regressão linear pode fornecer uma performance aceitável como linha de base, antes de considerar modelos mais complexos que podem exigir mais ajustes e validação.


Já sobre os fatores contra:

1. **Limitações na complexidade do relacionamento** -> A regressão linear assume uma relação linear entre as variáveis de entrada e a variável de saída. Se os dados possuem relações não-lineares complexas, um modelo de regressão linear pode não capturar essas nuances tão bem quanto modelos mais flexíveis.

2. **Sensibilidade a outliers** -> A presença de outliers nos dados pode influenciar negativamente os coeficientes estimados pela regressão linear, afetando a precisão das previsões.

3. **Assunções sobre os dados** -> A regressão linear faz certas suposições sobre os dados, como a independência das observações, normalidade dos erros e linearidade das relações. Se essas suposições não forem atendidas, a qualidade das previsões pode ser comprometida.

4. **Não adequado para problemas não-lineares complexos** -> Para problemas onde as relações entre variáveis são altamente não-lineares ou há interações complexas entre as variáveis, modelos mais avançados como árvores de decisão, SVMs ou redes neurais podem oferecer melhor desempenho.


Portanto, ao usar um modelo de regressão linear ao invés de qualquer outro modelo de regressão para prever notas IMDB, o objetivo de encontrar uma função que melhor se ajuste aos dados históricos para prever de forma precisa as notas de filmes não vistos anteriormente é atendido e comporta as limitações:.

In [93]:

# Carregar os dados
df = pd.read_csv('/content/desafio_indicium_imdb.csv')

# Preparar os dados para modelagem

 # Remover linhas com dados ausentes

df = df.dropna(subset=['IMDB_Rating'])

#Transformar variáveis preditoras em variáveis numéricas

df['Meta_score'] = pd.to_numeric(df['Meta_score'])
df['No_of_Votes'] = pd.to_numeric(df['No_of_Votes'])

df['Runtime'] = df['Runtime'].str.replace(" min", "")
df['Runtime'] = pd.to_numeric(df['Runtime'])

df['Gross'] = df['Gross'].str.replace(",", "")
df['Gross'] = pd.to_numeric(df['Gross'])

# Preencher valores ausentes nas variáveis preditoras com a média (ou outra estratégia)
df.fillna(df.select_dtypes(include=np.number).mean(), inplace=True)

# Dividir os dados em variáveis preditoras (X) e variável alvo (y)
X = df[['Meta_score', 'No_of_Votes','Runtime', 'Gross' ]]  # Variáveis preditoras (escolhidas pois tem o maior correlação de variáveis, visto no outro notebook)
y = df['IMDB_Rating']  # Variável alvo

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Salvar o modelo
with open('imdb_rating_model.pkl', 'wb') as f:
    pickle.dump(model, f)

As métricas utilizadas foram baseadas nas três primeiras variáveis mais relacionadas ao IMDB na fase exploratória dos dados.

Assim, a predição se torna mais rápida por descartar valores de magnitude despreziva e mais assertiva pois irá considerar os fatos de maior probabilidade.

In [94]:
# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

RMSE: 0.22939477393089167


O RMSE (Root Mean Squared Error) foi utilizado como métrica de avaliação porque mede a diferença média entre os valores previstos pelo modelo e os valores reais, sendo uma métrica popular para avaliar a precisão de modelos de regressão.

Algumas razões pelas quais o RMSE é uma escolha apropriada para avaliação são:

1. **Medida de erro** -> O RMSE mede a raiz quadrada da média dos quadrados dos erros entre as previsões do modelo e os valores reais. Isso fornece uma medida direta da dispersão dos erros, ou seja, o quão distantes as previsões estão dos valores verdadeiros de notas no IMDB.

2. **Sensibilidade a grandes erros** -> Como o RMSE envolve o quadrado dos erros, ele tende a penalizar mais fortemente grandes erros individuais em comparação com métricas como o MAE (Mean Absolute Error). Isso é importante em problemas onde erros grandes podem ter um impacto significativo na qualidade da previsão.

3. **Interpretabilidade** -> O RMSE é na mesma escala que a variável de saída (no nosso caso, a nota no IMDB), o que facilita a interpretação dos resultados em termos da precisão do modelo em relação às notas reais dos filmes.

4. **Facilidade de cálculo e comparação** -> O RMSE é amplamente utilizado na literatura e facilita a comparação entre diferentes modelos de regressão linear ou outros modelos preditivos. Isso ajuda na escolha do modelo que oferece melhor desempenho de previsão para o problema específico.


Logo,  pela natureza do problema de predição do IMDB,essa opção foi escolhida como avaliação do modelo.

# Previsão para o Filme Específico

In [95]:
# Características do filme "The Shawshank Redemption"

Predicted_Film = {'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': '28,341,469'}


Predicted_Film['Meta_score'] = pd.to_numeric(Predicted_Film['Meta_score'])
Predicted_Film['No_of_Votes'] = pd.to_numeric(Predicted_Film['No_of_Votes'])

Predicted_Film['Runtime'] = Predicted_Film['Runtime'].replace(" min", "")
Predicted_Film['Runtime'] = pd.to_numeric(Predicted_Film['Runtime'])

Predicted_Film['Gross'] = Predicted_Film['Gross'].replace(",", "")
Predicted_Film['Gross'] = pd.to_numeric(Predicted_Film['Gross'])

shawshank_features = np.array([Predicted_Film['Meta_score'], Predicted_Film['No_of_Votes'], Predicted_Film['Runtime'], Predicted_Film['Gross']])
shawshank_features = shawshank_features.reshape(1, -1)


# Carregar o modelo salvo
with open('imdb_rating_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Fazer a previsão
shawshank_rating_pred = loaded_model.predict(shawshank_features)
print(f"A nota prevista para o filme 'The Shawshank Redemption' é: {shawshank_rating_pred[0]}")


A nota prevista para o filme 'The Shawshank Redemption' é: 9.1049792465477


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Dado que o IMDB original do filme é 9,3 e a besa de dados não é exaustiva, o modelo conseguiu um desempenho bom para médio.